# 主題：天文照片辨認



        


            
        
    






## 組員
+ 蘇冠庭 師大物理四

+ 王博群 台大地理四

## 介紹
有在玩天文攝影，也有在一些FB社團或網頁中欣賞別人的作品，常常會看到有人丟一張照片上來詢問是什麼天體。然而實際情況是除非那個天體是星雲或是極為特別，否則若是沒有附近天區資訊的話就算是老手也不能很快解答。所以我就想到如果能提供很多天體的照片給程式訓練的話，辨識能力說不定能比人眼辨識來得好。

input: 天文照片（畫面有時會平移、旋轉、縮放，甚至影像的對比不同）

output: 天體名稱

通常會有辨認困難的會是星團，因為星雲或星系人們比較容易認得出來。 不過因為宇宙無窮無盡，可以先鎖定在比較有名的梅西爾天體中的星團照片進行訓練就好，甚至是改成星座辨認也不錯，星座一共 88 個，對一個人來說也是不好記完。

In [ ]:
## 實作方法
從 哈博官網、美國國家光學天文台、美國太空總署圖片庫、NASA搜尋 四個公開網站獲取資料
鎖定在梅西爾天體的共29個球狀星團，其編號是：2 3 4 5 9 10 12 13 14 15 19 22 28 30 53 54 55 56 62 68 69 70 71 72 75 79 80 92 107。
統一圖片的維度至100*100
增加樣本數，把現有的圖片用Imagrgenerator做影像處理（旋轉、縮放等等）每種星團額外增加100張照片。
用CNN進行辨識與訓練。

In [ ]:
# %load classifier.py
"""
Created on Sat Jun  8 11:39:20 2019

@author: Peter Wang
"""
from matplotlib.image import imread
from keras.models import Sequential  #用來啟動 NN
from keras.layers import Conv2D  # Convolution Operation
from keras.layers import MaxPooling2D # Pooling
from keras.layers import Flatten
from keras.layers import Dense # Fully Connected Networks
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
import os
import re
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread

# 设置生成器参数
PATH = 'Messier_data_jpg/'
SAVE_PATH = 'Test/'
datagen = ImageDataGenerator(featurewise_center=True,
                                   featurewise_std_normalization=True,
                                   rotation_range=15,
                                   zoom_range=0.2,
                                   rescale= 1/255,
                                   fill_mode='constant'
                                   )
gen_data = datagen.flow_from_directory(PATH, 
                                       batch_size=1, 
                                       shuffle=False, 
                                       save_to_dir=SAVE_PATH,
                                       save_prefix='M107_', 
				       target_size=(100, 100))
# 生成100张图
for i in range(100):
    gen_data.next() 

#Model
model = Sequential()  
model.add(Conv2D(32, (5, 5), strides = 3, input_shape = (100, 100, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (5, 5), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

# Third convolutional layer
model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(output_dim = 28, activation = 'relu'))
model.add(Dense(output_dim = 28, activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

#Training data
###y_train
path_train = os.listdir('Test')
names_list = []
for i in range(len(path_train)):
    name = int(re.findall('[0-9]+',path_train[i])[0])
    names_list.append(name)
x = list(set(names_list))
y_train = []
for i in range(len(names_list)):
    for j in range(len(x)):
        if names_list[i] == x[j]:
            y_train.append(j)
y_train = np.array(y_train)
y_train = np_utils.to_categorical(y_train,28)
np.save('y_train',y_train)

###x_train
x_train = imread('Test/%s'%path_train[0])
x_train = x_train.reshape(1,100,100,3)
for i in path_train:
    image = imread('Test/%s'%i)
    image = image.reshape(1,100,100,3)
    x_train = np.concatenate((x_train,image),axis=0)
x_train = np.delete(x_train,0,0)
np.save('x_train',x_train)

###y_test
path_test = os.listdir('Train')
test_names_list = []
for i in range(len(path_test)):
    name = int(re.findall('[0-9]+',path_test[i])[0])
    test_names_list.append(name)
index = 0
y_test_names = []
for i in path_test:
    image_path = os.listdir('Train/%s'%i)
    y_test = list(np.repeat(test_names_list[index],len(image_path)))
    y_test_names = y_test_names + y_test
    index = index+1
y_test = []
for i in range(len(y_test_names)):
    for j in range(len(x)):
        if y_test_names[i] == x[j]:
            y_test.append(j)
y_test = np.array(y_test)
y_test = np_utils.to_categorical(y_test,28)
np.save('y_test',y_test)
    
###x_test
path_x_test = os.listdir('Messier_data_resize')
x_test = np.load('Messier_data_resize/%s'%path_x_test[0])

for i in path_x_test:
    image = np.load('Messier_data_resize/%s'%i)
    x_test = np.concatenate((x_test,image),axis=0)
index_2 = [0,1,2]
x_test =np.delete(x_test,index_2,0)
np.save('x_test',x_test)

###標準化
x_train = x_train/255
x_test = x_test/255

###訓練
model.fit(x_train,y_train,batch_size=100,epochs=100)
model.save('messier_classifer_BC.h5')
    
    
###load_model
model.load_weights('messier_classifer_BC.h5')


###validation
def gallery():
    index = 1
    path_x_test = os.listdir('Messier object/')
    for i in path_x_test:
        image_paths2 = os.listdir('Messier object/%s'%i)
        for j in image_paths2:
            image = imread('Messier object/%s/%s'%(i,j))
            plt.subplot(10, 10 , index)
            plt.imshow(image)
            index = index +1
    print('圖片編號由左至右,再由上至下,共0~91')
def validation(testing):
    X = x_test[int(testing)]
    result = x[int(model.predict_classes(X.reshape(1,100,100,3)))]
    print(f'正確答案為M{y_test_names[int(testing)]}')
    print("模型預測為M%d"%result)


###validation accuracy
index_3 = 0
for i in range(92):
    X = x_test[int(i)]
    result = x[int(model.predict_classes(X.reshape(1,100,100,3)))]
    if y_test_names[int(i)] == result:
        index_3 = index_3+1
print(index_3/92)





        


            
        
    






## 結果
validation 的結果 accuracy 達到 91%
但是因為Train的資料是Test資料擴充而來 ， 因此原本就會預測有很高的正確率
如果有更多的野外拍攝照片可以更準確地進行validation

## 資料

In [ ]:
Training圖資 : https://drive.google.com/open?id=1QBJ2YyORCGy4ifLw4CXiQFo5b_HjO9Rl 
Test圖資 : https://drive.google.com/open?id=1-V24ekPO4xtETw7Z7ijtjfvvY45qdV_a